## Import

In [1]:
# 데이터 분석
import pandas as pd
import numpy as np

In [2]:
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [3]:
# 정규표현식
import re

In [4]:
# 형태소분석 okt
from konlpy.tag import Okt

## 파일 불러오기

In [28]:
df = pd.read_csv('review_data_2t.csv', encoding='utf-8')
df

,review,시설분류
0,?,문화
1,친절해요,문화
2,좋아요 만족스러워요,문화
3,와우~~,문화
4,굿,문화
...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원
141251,친절하십니다,동물병원
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원


## 함수

### 한글만 추출 함수

In [29]:
# 한글만 추출 + 2개이상 공백 1개로 변환
def extract_korean(text):
    # 한글만 추출
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]',' ',text)
    # 공백 2개 이상인 경우 1개로 변환
    text = re.sub(r'\s+',' ',text)
    return text


### 형용사 + 동사 추출 함수

In [30]:
# 형용사 추출하기
def get_adjectives(text):
    okt = Okt() # okt 객체 생성
    okt_pos = okt.pos(text)
    join_text = []
    for text, pos in okt_pos:
        if pos == 'Adjective' or pos == 'Verb':
            join_text.append(text)
    return " ".join(join_text)

## 한글만 추출 (review2 생성)

In [31]:
df['review2'] = df['review'].map(lambda review:extract_korean(review))

In [32]:
df.head(3)

,review,시설분류,review2
0,?,문화,
1,친절해요,문화,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요


## 파생컬럼 생성

In [5]:
# 시간이 오래 걸리는 작업시 시각적으로 진행상황 수치로( %) 보여준다.
from tqdm import tqdm
tqdm.pandas()

In [34]:
df['adj_verb'] = df['review'].progress_map(extract_korean).progress_map(get_adjectives)

100%|█████████████████████████████████████████████████████████████████████████| 141254/141254 [04:28<00:00, 526.78it/s]


In [37]:
df

,review,시설분류,review2,adj_verb
0,?,문화,,
1,친절해요,문화,친절해요,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
3,와우~~,문화,와우,
4,굿,문화,굿,
...,...,...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다
141251,친절하십니다,동물병원,친절하십니다,친절하십니다
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요


### 결측치 확인

In [38]:
df.isnull().sum()

review      0
시설분류        0
review2     0
adj_verb    0
dtype: int64

### 데이터의 값이 공백인거 NaN으로 변경

In [39]:
# 데이터값이 공백인거 NaN으로 변경
df_NanTest = df.replace(r'^\s*$', np.nan, regex=True)

In [40]:
df_NanTest

,review,시설분류,review2,adj_verb
0,?,문화,NaN,NaN
1,친절해요,문화,친절해요,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
3,와우~~,문화,와우,NaN
4,굿,문화,굿,NaN
...,...,...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다
141251,친절하십니다,동물병원,친절하십니다,친절하십니다
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요


In [41]:
df_NanTest.isnull().sum()

review          9
시설분류            0
review2      4927
adj_verb    30868
dtype: int64

### review2(한글만 남긴 리뷰)의 컬럼 데이터가 NaN이면 제거

In [42]:
df_NanTest.dropna(subset=['review2'], inplace=True)

In [43]:
df_NanTest.isnull().sum()

review          0
시설분류            0
review2         0
adj_verb    25941
dtype: int64

In [44]:
df_NanTest.dropna(subset=['adj_verb'], inplace=True)

In [45]:
df_NanTest.isnull().sum()

review      0
시설분류        0
review2     0
adj_verb    0
dtype: int64

In [46]:
df_NanTest.shape

(110386, 4)

## 저장

In [47]:
df_NanTest.to_csv('data/review_adj_verb.csv', index=False, encoding='utf-8-sig')

## 감성사전으로 라벨링 작업

### 데이터프레임 불러오기

In [6]:
df_review_data = pd.read_csv('data/review_adj_verb.csv', encoding='utf-8')

In [7]:
df_review_data.head(20)

,review,시설분류,review2,adj_verb
0,친절해요,문화,친절해요,친절해요
1,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
2,좋아요,문화,좋아요,좋아요
3,좋아요,문화,좋아요,좋아요
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,휴일날 찾아간 각화제 가을을 부르는 꽃이 피어있네요 ㅋㅋ,찾아간 부르는 피어있네요
5,산책하기 좋아요,문화,산책하기 좋아요,하기 좋아요
6,처음 가봤는데 산책로가 잘 되어있네요~\n한적하게 산책하며 힐링하기 좋습니다,문화,처음 가봤는데 산책로가 잘 되어있네요 한적하게 산책하며 힐링하기 좋습니다,가봤는데 잘 되어있네요 한적하게 하며 하기 좋습니다
7,보현사에서 시작~구릉산~갈매구릉산자락길~모다아울렛구리.갈매역까지 걷기\n\n갈매역 ...,문화,보현사에서 시작 구릉산 갈매구릉산자락길 모다아울렛구리 갈매역까지 걷기 갈매역 근처에...,있어 하기 좋음 있어 좋음
8,좋습니다,문화,좋습니다,좋습니다
9,좋아요,문화,좋아요,좋아요


### 정규화, 어간 - 추출 함수

In [8]:
def extract_stemming(words, tokenizer):
    # 문자열을 포함한 다양한 타입에서 NaN 확인
    if pd.isna(words):
        return

    # 전체 텍스트에 대해 한 번에 형태소 분석을 수행
    morphs = tokenizer.morphs(words, stem=True)
    
    # 결과에서 어간만 추출하여 조인
    return ' '.join(morph for morph in morphs if morph)

In [9]:
from konlpy.tag import Okt
okt = Okt()

In [10]:
from tqdm import tqdm
tqdm.pandas()
from functools import partial

In [12]:
# extract_stem 함수에 대한 partial 함수 생성
extract_stem_okt = partial(extract_stemming, tokenizer=okt)

In [13]:
df_review_data['adj_verb_stemmed'] = df_review_data['adj_verb'].progress_map(extract_stem_okt)

100%|█████████████████████████████████████████████████████████████████████████| 110386/110386 [01:54<00:00, 960.96it/s]


In [14]:
df_review_data

,review,시설분류,review2,adj_verb,adj_verb_stemmed
0,친절해요,문화,친절해요,친절해요,친절하다
1,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,좋다 만족스럽다
2,좋아요,문화,좋아요,좋아요,좋다
3,좋아요,문화,좋아요,좋아요,좋다
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,휴일날 찾아간 각화제 가을을 부르는 꽃이 피어있네요 ㅋㅋ,찾아간 부르는 피어있네요,찾아가다 부르다 피다
...,...,...,...,...,...
110381,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...,하다 가다 만지다 꼼꼼하다 살펴보다 먹다 되다 하다 맞다 오다 다행하다 좋다 지다
110382,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다,데려오다 세다 보다 드리다
110383,친절하십니다,동물병원,친절하십니다,친절하십니다,친절하다
110384,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요,꼼꼼하다 보다 챙기다 가다 따라가다 해주다 약바르다 있다 좋아지다


### 감성사전 불러오기

In [15]:
import json

In [16]:
with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    Sentiword_info = json.load(f)

In [17]:
sentiword_dic = pd.DataFrame(Sentiword_info)

In [18]:
sentiword_dic

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14849,반신반의하다,반신반의,0
14850,신비롭다,신비,1
14851,아리송하다,아리송,-1
14852,알쏭하다,알쏭하,-1


#### 감성사전 추가하기

In [19]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '친절해요', 'word_root': '친절', 'polarity' : '2'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

### 라벨링 작업

In [24]:
# 감정사전 key-value(단어 - 긍정부정값)
hash_map = {}

In [25]:
# 감정사전 key-value 값 해싱
for index, row in sentiword_dic.iterrows():
  hash_map[row['word']] = row['polarity']

In [26]:
# 리뷰값 불러와서 긍정 부정 분석
def calculate_sentiment_korean(text):

    score = 0
    words = text.split()
    for word in words:
      score += int(hash_map.get(word,0))

    if score > 0: return 1
    elif  score < 0 : return -1
    else : return 0

In [31]:
def calculate_sentiment_score(text):

    score = 0
    words = text.split()
    for word in words:
      score += int(hash_map.get(word,0))
    return score

In [28]:
# 리뷰 데이터에 대한 감정 라벨링
from tqdm import tqdm
tqdm.pandas()
df_review_data['label'] = df_review_data['adj_verb_stemmed'].apply(calculate_sentiment_korean)

In [32]:
# 리뷰 데이터에 대한 감정 라벨링
from tqdm import tqdm
tqdm.pandas()
df_review_data['score'] = df_review_data['adj_verb_stemmed'].apply(calculate_sentiment_score)

In [33]:
df_review_data

,review,시설분류,review2,adj_verb,adj_verb_stemmed,label,score
0,친절해요,문화,친절해요,친절해요,친절하다,1,2
1,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,좋다 만족스럽다,1,4
2,좋아요,문화,좋아요,좋아요,좋다,1,2
3,좋아요,문화,좋아요,좋아요,좋다,1,2
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,휴일날 찾아간 각화제 가을을 부르는 꽃이 피어있네요 ㅋㅋ,찾아간 부르는 피어있네요,찾아가다 부르다 피다,0,0
...,...,...,...,...,...,...,...
110381,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...,하다 가다 만지다 꼼꼼하다 살펴보다 먹다 되다 하다 맞다 오다 다행하다 좋다 지다,1,2
110382,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다,데려오다 세다 보다 드리다,0,0
110383,친절하십니다,동물병원,친절하십니다,친절하십니다,친절하다,1,2
110384,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요,꼼꼼하다 보다 챙기다 가다 따라가다 해주다 약바르다 있다 좋아지다,1,3


In [34]:
df_review_data.to_csv('data/knu_data_review_test2.csv', index=False, encoding='utf-8-sig')

In [36]:
df_review_data.to_excel('data/knu_data_review_test2.xlsx', index=False)